## 5.6 Carsy

Aplikacja wykorzystuje `BottomNavigation` oraz `DrawerNavigation`. Po raz kolejny wykorzystamy tylko statyczne dostarczone przez obiekt `dataProvider`. Wykorzystamy `RecyclerView` w kilku różnych konfiguracjach. Będzie to uboga wersja [Fuelio](https://play.google.com/store/apps/details?id=com.kajda.fuelio&hl=pl&gl=US)
- `RecyclerView` z grupowaniem elementów osiągniętym przez wykorzystanie dwóch `ViewHolder`

### **BottomNavigation**

Rozpocznijmy od dodania trzech pustych fragmentów (`OverviewFragment`, `CalculatorsFragment`, `TimeLineFragment`) i nawigacji

`navigation.xml`
```xml
<?xml version="1.0" encoding="utf-8"?>
<navigation xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:id="@+id/navigation"
    app:startDestination="@id/overviewFragment">

    <fragment
        android:id="@+id/calculatorsFragment"
        android:name="pl.udu.uwr.pum.carsyappkotlin.fragments.CalculatorsFragment"
        android:label="fragment_calculators"
        tools:layout="@layout/fragment_calculators" />
    <fragment
        android:id="@+id/timeLineFragment"
        android:name="pl.udu.uwr.pum.carsyappkotlin.fragments.TimeLineFragment"
        android:label="fragment_time_line"
        tools:layout="@layout/fragment_time_line" />
    <fragment
        android:id="@+id/overviewFragment"
        android:name="pl.udu.uwr.pum.carsyappkotlin.fragments.OverviewFragment"
        android:label="fragment_overview"
        tools:layout="@layout/fragment_overview" />
</navigation>
```

Po raz kolejny dodajemy tylko fragmenty nie definiując dla nich akcji. `OverviewFragment` oznaczam jako fragment domowy. Musimy również utworzyć `menu` dla `BottomNavigation`

`bottom_menu.xml`
```xml
<?xml version="1.0" encoding="utf-8"?>
<menu xmlns:android="http://schemas.android.com/apk/res/android">
    <item
        android:id="@id/overviewFragment"
        android:title="@string/overview" />
    <item
        android:id="@id/timeLineFragment"
        android:title="@string/time_line" />

    <item
        android:id="@id/calculatorsFragment"
        android:title="@string/calculators" />
</menu>
```

Pamiętajmy, że `android:id` muszą być takie same jak w `navigation.xml` aby automatyczna nawigacja działała prawidłowo.

Przejdźmy do `MainActivity` i dodajmy nawigację oraz połączmy ją z `BottomNavigation`

In [ ]:
class MainActivity : AppCompatActivity() {

    private val navController: NavController by lazy {
        val navHostFragment = supportFragmentManager.findFragmentById(R.id.nav_host_fragment)
                as NavHostFragment
        navHostFragment.findNavController()
    }

    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContentView(R.layout.activity_main)

        val bottomNav = findViewById<BottomNavigationView>(R.id.bottom_nav_view)
        bottomNav?.setupWithNavController(navController)
    }
}

W efekcie powinniśmy otrzymać podstawową nawigację z trzema ekranami.

<img src="https://media1.giphy.com/media/x0j1SX0qmejSUNRtHu/giphy.gif?cid=790b7611bcaddd3fe77b775f20dd38e5e0cfa788c1e83df7&rid=giphy.gif&ct=g" width="150" />

### **TimeLineFragment**

Chcemy przygotować listę w postaci osi czasu, która będzie zbliżona do tego co oferuje aplikacja **Fuelio**

<img src="https://play-lh.googleusercontent.com/Lm6SO1jsUF-VjsrfkIV9x-qpPIkIOIvgWkBGmxyZuHWPSbkYMS0oLhgoBk9wLXq6Xw=w5120-h2880" width="150" />

Zrobimy to w postaci `RecyclerView` z dwoma `ViewHolder`. Wpierw przygotujmy dane. Typ kosztu zdefiniujemy jako `enum class`, będziemy przechowywać w niej typ oraz ikonę.

In [ ]:
enum class CostType(val costType: String, val icon: Int) {
    REFUELING("Tankowanie", R.drawable.ic_fuel),
    SERVICE("Serwis", R.drawable.ic_car_repair),
    PARKING("Parking", R.drawable.ic_parking),
    INSURANCE("Ubezpieczenie", R.drawable.ic_general_cost),
    TICKET("Mandat", R.drawable.ic_ticket)
}

Ikony standardowo wybieramy spośród dostępnych w **Android Studio** (**New -> Vector Asset**). Model danych zawiera typ kosztu, datę oraz kwotę.

In [ ]:
data class Cost (
    val type: CostType,
    val date: LocalDate,
    val amount: Int
)

Ponieważ będziemy posiadać pojedynczy `RecyclerView` i dwa `ViewHolder`, musimy w jakiś sposób rozróżnić typ naszych danych - dla uproszczenia będziemy sortować po miesiącach (bez uwzględnienia lat). Więc chcemy zdefiniować dwa typy 
- `CostListItem` - klasa bazowa

In [ ]:
open class CostListItem(
    val type: Int
) {
    companion object {
        const val TYPE_DATE = 0
        const val TYPE_GENERAL = 1
    }
}

- `CostDateItem` - posiadający tylko datę po której będziemy sortować oraz na podstawie której utworzymy wersję layoutu dla `RecyclerView`

In [ ]:
class CostDateItem(
    val date: String
) : CostListItem(TYPE_DATE)

- `CostGeneralItem` - każdy inny element dla którego również utworzymy layout

In [ ]:
class CostGeneralItem(
    val cost: Cost
) : CostListItem(TYPE_GENERAL)

Przejdźmy do obiektu `DataProvider` - w pierwszym kroku utworzymy listę kosztów

In [ ]:
private val generalCosts = List(100) {
    Cost(
        CostType.values()[Random.nextInt(CostType.values().size)],
        LocalDate.of(2022, Random.nextInt(1,13), Random.nextInt(1,28)),
        Random.nextInt(5000)
    )
}

Ponieważ mamy jeden `RecyclerView`, więc będziemy mieć jedną listę - musimy ją odpowiednio przygotować. Wykorzystamy `Map` i posortujemy całość po miesiącach, następnie pogrupujemy względem miesięcy.

In [ ]:
private val groupedCostMap: Map<String, List<Cost>> = generalCosts
    .sortedBy { it.date.month }
    .groupBy { it.date.month.toString() }

Następnie napiszemy funkcję `getTimeLineList` zwracająca listę dla `RecyclerView`

In [ ]:
fun getTimeLineList(): List<CostListItem>{

Utwórzmy listę pomocniczą

In [ ]:
val list = mutableListOf<CostListItem>()

Przechodzimy po wszystkich kluczach w naszej mapie

In [ ]:
for (date: String in groupedCostMap.keys) {

W pętli tworzę listę wszystkich elementów z danego miesiąca, następnie ją sortuję względem dnia miesiąca i każdy element `CostGeneralItem` dodaję do `list`

In [ ]:
val groupItems: List<Cost>? = groupedCostMap[date]
groupItems?.sortedBy { it.date.dayOfMonth }?.forEach {
    list.add(CostGeneralItem(it))
}

Ostatnim elementem pętli będzie dodanie `CostDateItem`

In [ ]:
list.add(CostDateItem(date))

Na koniec wychodzimy z pętli `for` i zwracamy listę

In [ ]:
    }
    return list
}

Pełna funkcja

In [ ]:
fun getTimeLineList(): List<CostListItem>{
    val list = mutableListOf<CostListItem>()
    for (date: String in groupedCostMap.keys) {
        val groupItems: List<Cost>? = groupedCostMap[date]
        groupItems?.sortedBy { it.date.dayOfMonth }?.forEach {
            list.add(CostGeneralItem(it))
        }
        list.add(CostDateItem(date))
    }
    return list
}

Podsumowując:
- tworzą listę do której wrzucę wszystkie elementy odpowiednio posortowane - lista zawiera elementy typu `CostListItem`
- przechodzę po kluczach wcześniej utworzonej mapy (miesiące)
- dodaję wszystkie elementy `CostGeneralItem` - wszystkie koszty dla danego miesiąca
- na koniec dodaję `CostDateItem` - miesiąc

Czyli tworzymy listę w odwrotnej kolejności - wpierw mamy wszystkie elementy dla danego miesiąca, po nim znajduje się element oznaczający dany miesiąc.

Przejdźmy do `TimeLineAdapter` - klasa będzie przyjmowała w parametrze `Context` i rozszerzała `RecyclerView.Adapter`

In [ ]:
class TimeLineAdapter(val context: Context) 
    : RecyclerView.Adapter<RecyclerView.ViewHolder>() {}

W pierwszej kolejności dodajmy listę elementów, ponieważ metoda `getTimeLineList` zwraca listę w odwrotnej kolejności, tutaj w pierwszym kroku wykonamy `reversed`

In [ ]:
private val itemList = DataProvider.getTimeLineList().reversed()

Zdefiniujmy dwie klasy `ViewHolder` - pierwszą będzie klasa dla elementu grupującego (tutaj jest to data - miesiąc)

In [ ]:
inner class DateViewHolder(view: View) : RecyclerView.ViewHolder(view) {

    private val dateTextView: TextView = view.findViewById(R.id.timeLineDateTextView)

    fun bind(item: CostDateItem) {
        dateTextView.text = item.date
    }
}

Dla tego `ViewHolder` definiujemy layout w pliku `date_item_timeline.xml`


```xml
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:layout_marginStart="4dp"
    android:layout_marginEnd="4dp">

    <View
        android:id="@+id/RVcolorBarView"
        android:layout_width="16px"
        android:layout_height="0dp"
        android:layout_marginStart="24dp"
        android:background="?attr/colorPrimary"
        android:outlineAmbientShadowColor="?attr/colorPrimary"
        android:outlineSpotShadowColor="?attr/colorPrimary"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <ImageView
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="19dp"
        android:layout_marginTop="24dp"
        android:background="@drawable/ic_round_circle"
        android:backgroundTint="?attr/colorPrimary"
        android:contentDescription="@string/bullet"
        app:layout_constraintBottom_toBottomOf="@+id/RVcolorBarView"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="@+id/RVcolorBarView" />

    <TextView
        android:id="@+id/timeLineDateTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="16dp"
        android:layout_marginTop="24dp"
        android:fontFamily="serif"
        android:textColor="?attr/colorSecondary"
        android:padding="4dp"
        android:text="@string/date"
        android:textSize="16sp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintStart_toStartOf="@id/RVcolorBarView"
        app:layout_constraintTop_toTopOf="parent" />

</androidx.constraintlayout.widget.ConstraintLayout>
```

Jak widzimy mamy tylko jedno pole `TextView` wyświetlające nazwę miesiąca po którym grupujemy. Pozostałe dwa elementy są graficznym indykatorem - `View` jest linią o zadanej grubości i `ImageView` tutaj ma formę wypełnionego koła.

Zdefiniujmy również drugi `ViewHolder` dla pozostałych elementów na liście - tutaj chcemy pokazać typ kosztu, kwotę, pełną datę, oraz ikonę dla odpowiedniego typu kosztu.

In [ ]:
inner class GeneralViewHolder (view: View) : RecyclerView.ViewHolder(view) {

    private val costTypeTextView: TextView = view.findViewById(R.id.timeLineCostTypeNameTextView)
    private val fullDateTextView: TextView = view.findViewById(R.id.timeLineFullDateTextView)
    private val amountTextView: TextView = view.findViewById(R.id.timeLineCostAmountTextView)
    private val iconImageView: ImageView = view.findViewById(R.id.iconTimeLineImageView)

    fun bind(item: CostGeneralItem) {
        costTypeTextView.text = item.cost.type.costType
        fullDateTextView.text = item.cost.date.format(dateFormatter)
        (item.cost.amount.toString() + " zł").also { amountTextView.text = it }
        iconImageView.background = ContextCompat.getDrawable(context, item.cost.type.icon)
    }
}

Będziemy również formatować nieco `String` przy dacie, więc zdefiniujmy `DateTimeFormatter`

In [ ]:
val dateFormatter: DateTimeFormatter = DateTimeFormatter.ofPattern("yyyy MMM dd")

Layout jest nieco bardziej skomplikowany

```xml
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:layout_marginStart="4dp"
    android:layout_marginEnd="4dp">

    <View
        android:id="@+id/RVcolorBarView"
        android:layout_width="16px"
        android:layout_height="0dp"
        android:layout_marginStart="24dp"
        android:background="?attr/colorPrimary"
        android:outlineAmbientShadowColor="?attr/colorPrimary"
        android:outlineSpotShadowColor="?attr/colorPrimary"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <ImageView
        android:id="@+id/imageView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="3.2dp"
        android:background="@drawable/ic_round_circle_big"
        android:backgroundTint="?attr/colorPrimary"
        android:contentDescription="@string/bullet"
        app:layout_constraintBottom_toBottomOf="@+id/RVcolorBarView"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="@+id/RVcolorBarView" />

    <ImageView
        android:id="@+id/iconTimeLineImageView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="12dp"
        android:layout_marginTop="8dp"
        android:background="@drawable/ic_car_repair"
        android:backgroundTint="@color/white"
        android:contentDescription="@string/bullet"
        app:layout_constraintBottom_toBottomOf="@+id/imageView"
        app:layout_constraintStart_toStartOf="@+id/imageView"
        app:layout_constraintTop_toTopOf="@+id/imageView"
        app:layout_constraintVertical_bias="0.285" />

    <TextView
        android:id="@+id/timeLineCostTypeNameTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:padding="4dp"
        android:text="@string/costtype_name"
        android:textSize="24sp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintStart_toEndOf="@+id/imageView"
        app:layout_constraintTop_toTopOf="parent" />

    <TextView
        android:id="@+id/timeLineCostAmountTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginEnd="16dp"
        android:padding="4dp"
        android:text="1200 zł"
        android:textSize="24sp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <TextView
        android:id="@+id/timeLineFullDateTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:fontFamily="serif"
        android:paddingStart="4dp"
        android:text="@string/date"
        android:textSize="16sp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintStart_toEndOf="@+id/imageView"
        app:layout_constraintTop_toBottomOf="@+id/timeLineCostTypeNameTextView" />

</androidx.constraintlayout.widget.ConstraintLayout>
```

Zauważmy że znów mamy `View` o tych samych parametrch co w poprzednim layoucie co sprawia wrażenie ciągłości przez wszystkie elementy. Mamy również dwa `ImageView` - pierwszy zastosowałem jako okrągłe tło o jednolitym kolorze dla ikony obecnej w drugim.

Przejdźmy do metody `onCreateViewHolder` - zwrócimy `ViewHolder` dla zadanego `viewType`. Ponieważ będziemy wykorzystywać dwa, musimy zdefiniować jeszcze `viewType`.

In [ ]:
override fun onCreateViewHolder(parent: ViewGroup, viewType: Int): RecyclerView.ViewHolder {
    return when (viewType) {
        CostListItem.TYPE_DATE ->
            DateViewHolder(LayoutInflater.from(parent.context).inflate(
                R.layout.date_item_timeline, parent, false
            ))
        else ->
            GeneralViewHolder(LayoutInflater.from(parent.context).inflate(
                R.layout.general_item_timeline, parent, false
            ))
    }
}

Aby zdefiniować `viewType` musimy nadpisać metodę `getItemViewType`

In [ ]:
override fun getItemViewType(position: Int): Int = itemList[position].type // zwraca typ elementu na liście

Zaimplementujmy metodę `getItemCount`

In [ ]:
override fun getItemCount(): Int = itemList.size

Pozostaje implementacja metody `onBindViewHolder` - w zależności od typu `ViewHolder` chcemy przekazać `CostListItem` jako `CostGeneralItem` zawierający instancję `Cost` lub `CostDateItem` zawierający nazwę miesiąca jako `String`.

In [ ]:
override fun onBindViewHolder(holder: RecyclerView.ViewHolder, position: Int) {
    when (holder.itemViewType) {
        CostListItem.TYPE_DATE -> (holder as DateViewHolder).bind(
            item = itemList[position] as CostDateItem
        )
        CostListItem.TYPE_GENERAL -> (holder as GeneralViewHolder).bind(
            item = itemList[position] as CostGeneralItem
        )
    }
}

Przejdźmy do `TimeLineFragment` i dodajmy nasz RecyclerView

In [ ]:
override fun onViewCreated(view: View, savedInstanceState: Bundle?) {
    super.onViewCreated(view, savedInstanceState)
    view.findViewById<RecyclerView>(R.id.timeLineRecyclerView).apply {
        layoutManager = LinearLayoutManager(context)
        adapter = TimeLineAdapter(context)
    }
}

Możemy przetestować aplikację

<img src="https://media4.giphy.com/media/S4iOJk9GSVcizP2tRw/giphy.gif?cid=790b76113b3cee84bee0e3484c2fbc03807cf26e022c376a&rid=giphy.gif&ct=g" width="150" />